# Parameters

The computational and physical parameters are handled by `class xcompact3d_toolbox.Parameters`. It is built on top of [Traitlets](https://traitlets.readthedocs.io/en/stable/index.html), which aims to make the parameters compatible with what xcompact3d expects, and also brings some advantages:

* Attributes are type-checked;
* Default values, restrictions and conections between related parameters are applied where necessary;
* 'On change' callbacks for validation and observation;
* Two-way linking with [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/).

In [1]:
import numpy as np

import xcompact3d_toolbox as x3d

The first step is to establish numerical precision. Use `np.float64` if Xcompact3d was compiled with the flag `-DDOUBLE_PREC` (check the Makefile), use `np.float32` otherwise:

In [2]:
x3d.param["mytype"] = np.float32

## Initialization

There are a few ways to initialize the class. First, calling it with no arguments initializes all variables with default value:

In [3]:
prm = x3d.Parameters()

You can access a list with all the available variables at the [Api reference](https://xcompact3d-toolbox.readthedocs.io/en/latest/Docstrings.html).

Let's see how it looks like:

In [4]:
prm

! -*- mode: f90 -*-

!===================
&BasicParam
!===================

           beta = 1.0             ! Refinement parameter
             dt = 0.001           ! Time step
          gravx = 0.0             ! Gravity unitary vector in x-direction
          gravy = 0.0             ! Gravity unitary vector in y-direction
          gravz = 0.0             ! Gravity unitary vector in z-direction
         ifirst = 0               ! The number for the first iteration
           iibm = 0               ! Flag for immersed boundary method (0: No, 1: Yes)
            iin = 0               ! Defines pertubation at initial condition
          ilast = 0               ! The number for the last iteration
        ilesmod = 0               ! Enables Large-Eddy methodologies (0: No, 1: Yes)
   inflow_noise = 0.0             ! Turbulence intensity (1=100%) !! Inflow condition
     init_noise = 0.0             ! Turbulence intensity (1=100%) !! Initial condition
          ipost = 0               ! E

It is possible to access and/or set values afterwards:

In [5]:
# Reynolds Number
print(prm.re)

# attribute new value
prm.re = 1e6
print(prm.re)

1000.0
1000000.0


Second, we can specify some values, and let the missing ones be initialized with default value:

In [6]:
prm = x3d.Parameters(
    filename="example.i3d",
    itype=10,
    nx=129,
    ny=65,
    nz=32,
    xlx=15.0,
    yly=10.0,
    zlz=3.0,
    nclx1=2,
    nclxn=2,
    ncly1=1,
    nclyn=1,
    nclz1=0,
    nclzn=0,
    iin=1,
    istret=2,
    re=300.0,
    init_noise=0.0125,
    inflow_noise=0.0125,
    dt=0.0025,
    ifirst=1,
    ilast=45000,
    irestart=0,
    icheckpoint=45000,
    ioutput=200,
    iprocessing=50,
)

It is easy to write `example.i3d` to disc, just type:

In [7]:
prm.write()

And finally, it is possible to read the parameters from the disc:

In [8]:
prm = x3d.Parameters(filename="example.i3d")
prm.load()

The same result is obtained in a more concise way:

In [9]:
prm = x3d.Parameters(loadfile="example.i3d")

The class can also read the previous parameters format ([se more information here](https://github.com/fschuch/xcompact3d_toolbox/issues/7)):

``` python
prm = x3d.Parameters(loadfile="incompact3d.prm")
```

There are extra methods to:

* Read a binary field from Xcompact3d:

    ``` python
    prm.read_field('filename')
    ```

* Read several binary files at once:

    ``` python
    prm.read_all_fields('filename_pattern')
    ```

* Write `xdmf` files, so the binary files can be open in any external visualization tool:

    ``` python
    prm.write_xdmf()
    ```

* Compute the coordinates, including support for mesh refinement in y:

In [10]:
prm.get_mesh

{'x': array([ 0.       ,  0.1171875,  0.234375 ,  0.3515625,  0.46875  ,
         0.5859375,  0.703125 ,  0.8203125,  0.9375   ,  1.0546875,
         1.171875 ,  1.2890625,  1.40625  ,  1.5234375,  1.640625 ,
         1.7578125,  1.875    ,  1.9921875,  2.109375 ,  2.2265625,
         2.34375  ,  2.4609375,  2.578125 ,  2.6953125,  2.8125   ,
         2.9296875,  3.046875 ,  3.1640625,  3.28125  ,  3.3984375,
         3.515625 ,  3.6328125,  3.75     ,  3.8671875,  3.984375 ,
         4.1015625,  4.21875  ,  4.3359375,  4.453125 ,  4.5703125,
         4.6875   ,  4.8046875,  4.921875 ,  5.0390625,  5.15625  ,
         5.2734375,  5.390625 ,  5.5078125,  5.625    ,  5.7421875,
         5.859375 ,  5.9765625,  6.09375  ,  6.2109375,  6.328125 ,
         6.4453125,  6.5625   ,  6.6796875,  6.796875 ,  6.9140625,
         7.03125  ,  7.1484375,  7.265625 ,  7.3828125,  7.5      ,
         7.6171875,  7.734375 ,  7.8515625,  7.96875  ,  8.0859375,
         8.203125 ,  8.3203125,  8.4375   ,

More details about I/O and array manipulations with [xarray](http://xarray.pydata.org/en/stable/index.html) will be included in a new tutorial.

## Traitlets

### Type-checking

All parameters are type-checked, to make sure that they are what `Xcompact3d` expects. Use the cellcode below to see how a `TraitError` pops out when we try:

```python
prm.itype = 10.5
prm.itype = -5
prm.itype = 12
prm.itype = 'sandbox'

```


### Validation

Some parameters, like mesh points (`nx`, `ny` and `nz`), trigger a validation operation when a new value is attributed to them.
Due to restrictions at the FFT library, they must be equal to:

$$
n_i = \left\{ \begin{array}{ll} 2^{1+a} \times 3^b \times 5^c &\mbox{if periodic,} \\ 2^{1+a} \times 3^b \times 5^c + 1 &\mbox{otherwise,}
\end{array} \right.
$$

where $a$, $b$ and $c$ are non negative integers. In addition, the derivatives stencil imposes that:

$$
n_i \ge \left\{ \begin{array}{ll} 8 &\mbox{if periodic,} \\ 9 &\mbox{otherwise.}
\end{array} \right.
$$

Again, give it a try at the cellcode below:

```python
prm.nx = 129
prm.nx = 4
prm.nx = 60
prm.nx = 61
```

### Observation

Other parameters, like mesh resolution (`dx`, `dy` and `dz`), are automaticaly updated when any new atribution occurs to mesh points and/or domain size. Let's create a quick print functions to play with:

In [11]:
def show_param():
    for var in "nclx1 nclxn nx xlx dx".split():
        print(f"{var:>5} = {getattr(prm, var)}")

We are starting with:

In [12]:
show_param()

nclx1 = 2
nclxn = 2
   nx = 129
  xlx = 15.0
   dx = 0.1171875


Let's change just the domain's length:

In [13]:
prm.xlx = 50.0

show_param()

nclx1 = 2
nclxn = 2
   nx = 129
  xlx = 50.0
   dx = 0.390625


The resolution was updated as well. Now the number of mesh points:

In [14]:
prm.nx = 121

show_param()

nclx1 = 2
nclxn = 2
   nx = 121
  xlx = 50.0
   dx = 0.4166666666666667


Again, the resolution was updated. Now we set a new mesh resolution, this time, `xlx` will be updated in order to satisfy the new resolution:

In [15]:
prm.dx = 1e-2

show_param()

nclx1 = 2
nclxn = 2
   nx = 121
  xlx = 1.2
   dx = 0.01


Boundary conditions are observed as well. Xcompact3d allows three different BC for velocity:

* Periodic `0`;
* Free-slip `1`;
* Dirichlet `2`.

They can be assigned individualy for each of the six boundaries:

* `nclx1` and `nclxn`, where $x=0$ and $x=xlx$;
* `ncly1` and `nclyn`, where $y=0$ and $y=yly$;
* `nclz1` and `nclzn`, where $z=0$ and $z=zlz$.

It leads to 5 possibilities (`00`, `11`, `12`, `21` and `22`), because both boundary must be periodic, or not, so `0` cannot be combined.

Let's check it out, we are starting with:

In [16]:
show_param()

nclx1 = 2
nclxn = 2
   nx = 121
  xlx = 1.2
   dx = 0.01


We will change just one side to periodic (`nclx1 = 0`), for consistence, the other side should be periodic too. Let's see:

In [17]:
prm.nclx1 = 0

show_param()

nclx1 = 0
nclxn = 0
   nx = 120
  xlx = 1.2
   dx = 0.01


Now free-slip in one side (`nclx1 = 1`), and the other should be non-periodic:

In [18]:
prm.nclx1 = 1

show_param()

nclx1 = 1
nclxn = 1
   nx = 121
  xlx = 1.2
   dx = 0.01


Setting the other boundary to periodic:

In [19]:
prm.nclxn = 0

show_param()

nclx1 = 0
nclxn = 0
   nx = 120
  xlx = 1.2
   dx = 0.01


and now back to Dirichlet:

In [20]:
prm.nclxn = 2

show_param()

nclx1 = 2
nclxn = 2
   nx = 121
  xlx = 1.2
   dx = 0.01


This time, free-slip:

In [21]:
prm.nclxn = 1

show_param()

nclx1 = 2
nclxn = 1
   nx = 121
  xlx = 1.2
   dx = 0.01


There was no need to update `nclx1`, because `1` and `2` can be freely combined. Notice that `nx` was modified properly from 121 to 120 and then back, according to the possible values, `dx` and `xlx` stayed untouched.

### Metadata

Traitlets types constructors have a `tag` method to store metadata in a dictionary. In the case of Xcompact3d-toolbox, two are especially useful:

* `group` defines to what namespace a given parameter belongs when the class is written to `.i3d` file (`.write()` method) or read from `.i3d` or `.prm` files (`.load()` method), parameters without a group are ignored for both methods;
* `desc` contains a brief description of each parameter that is shown on screen as we saw above, and also printed with the `.write()` method.

### Declaring new parameters

You probably would like to add more parameters for your own flow configuration, or because some of them were not implemented yet (it is a work in progress).

To do so, any auxiliar varible can be included after initialization, like:

In [22]:
prm.my_variable = 0  # or any other datatype

It was called auxiliar variable because, in this way, it will be available only for the Python aplication.

In order to include it at the `.i3d` file and make it available for Xcompact3d, we can create a subclass that inherits all the functionality from `xcompact3d_toolbox.Parameters`:

In [23]:
import traitlets


# Create a class named my_Parameters, which inherits the properties all properties and methods
class my_Parameters(x3d.Parameters):
    # .tag with group and description guarantees that the new variable will
    # be compatible with all functionalities (like .write() and .load())
    a_my_variable = traitlets.Int(default_value=0, min=0).tag(
        group="BasicParam", desc="An example at the Tutorial <------"
    )

    # And a custom method, for instance
    def my_method(self):
        return self.a_my_variable * 2


prm = my_Parameters(nx=257, ny=129, nz=31, a_my_variable=10)  # and here we go

# Testing the method
print(prm.my_method())

# Show all parameters on screen
prm

20


! -*- mode: f90 -*-

!===================
&BasicParam
!===================

  a_my_variable = 10              ! An example at the Tutorial <------
           beta = 1.0             ! Refinement parameter
             dt = 0.001           ! Time step
          gravx = 0.0             ! Gravity unitary vector in x-direction
          gravy = 0.0             ! Gravity unitary vector in y-direction
          gravz = 0.0             ! Gravity unitary vector in z-direction
         ifirst = 0               ! The number for the first iteration
           iibm = 0               ! Flag for immersed boundary method (0: No, 1: Yes)
            iin = 0               ! Defines pertubation at initial condition
          ilast = 0               ! The number for the last iteration
        ilesmod = 0               ! Enables Large-Eddy methodologies (0: No, 1: Yes)
   inflow_noise = 0.0             ! Turbulence intensity (1=100%) !! Inflow condition
     init_noise = 0.0             ! Turbulence intens

Take a look at the source code of [parameters.py](https://github.com/fschuch/xcompact3d_toolbox/blob/master/xcompact3d_toolbox/param.py) if you need more examples for different datatypes.

## Graphical User Interface

<div class="alert alert-info">

For an interactive experience [launch this tutorial on Binder](https://mybinder.org/v2/gh/fschuch/xcompact3d_toolbox/master?urlpath=lab/tree/docs/tutorial), the widgets are not so responsive when disconnected from a Python application.

</div>

To conclude this part of the tutorial, let's see another option to handle the parameters. The class `ParametersGui` is a subclass of `Parameters`, and includes all the features described above. In addition, `ParametersGui` offers an user interface with [IPywidgets](https://ipywidgets.readthedocs.io/en/stable/index.html).

It is still under development, more parameters and features are going to be included soon, as well as more widgets.

Just like before, we start with:

In [24]:
prm = x3d.ParametersGui()

Widgets are returned on demand when any instance of `class ParametersGui` is called, let’s see:

In [25]:
prm("nx", "xlx", "dx", "nclx1", "nclxn")

You can play around with the widgets above and see the effect of the observations made previously.

Notice that the [Traitlets](https://traitlets.readthedocs.io/en/stable/index.html) parameters are related to the value at their widgets in a **two-way link**, in this way, a print will show the actual value on the widgets:

In [26]:
show_param()

nclx1 = 2
nclxn = 2
   nx = 17
  xlx = 1.0
   dx = 0.0625


Give it a try, modify the values at the widgets and print them again.

It also works on the other way, set a new value to a parameters will change its widget, try it:

In [27]:
#prm.nclx1 = 0

And of course, different widgets for the same parameter are always synchronized, change the widget below and see what happens with the widget above:

In [28]:
prm('nx')

A last example is about the domain decomposition for parallel computation, Xcompact3d uses [2DECOMP&FFT](http://www.2decomp.org/).
The available options for `p_row` and `p_col` are presented as functions of the number of computational cores `ncores`, notice that `p_row * p_col = ncores` should be respected and `p_row * p_col = 0` activates the auto-tunning mode. The widgets are prepared to respect these restrictions:

In [29]:
prm('ncores', 'p_row', 'p_col')

To conclude this part of the tutorial, let’s see what happend when `class ParametersGui` is presented on screen, hover the mouse over some variable to see its description:

In [30]:
prm